In [70]:
import argparse
import jinja2
import logging
import MDAnalysis as mda
import numpy as np
from pdb2pqr import run_pdb2pqr
import propka.run as pkrun
import itertools
import os
#from math import *
#import ipywidgets as widgets
#from IPython.display import display

In [71]:
os.chdir('/Users/anze/Git/pdb2xyz/tests')

### Get protonation states via propka

In [87]:
PDB_id='4F5S'
pdb_file='%s.pdb' % PDB_id

In [88]:
propka.run.single(pdb_file)

In [95]:
# obtain a list of partial charges based on propka-determined pKa values
def partial_charges(propka_file,pH):

    with open(propka_file) as fp:
        result = list(itertools.takewhile(lambda x: '---' not in x, 
            itertools.dropwhile(lambda x: 'Group' not in x, fp)))

    result=result[1:]
    result=np.array([line.split()[:-1] for line in result])

    negative=['C-','ASP','GLU','TYR','CYS']
    positive=['N+','ARG','LYS','HIS']

    pqr=result.copy()

    for i,entry in enumerate(result):

        AA=entry[0]
        pKa=float(entry[-1])

        if AA in negative: pqr[i,-1] = '%.6f' % (- 10**(pH-pKa) / (1 + 10**(pH-pKa)))
        elif AA in positive: pqr[i,-1] = '%.6f' % (1.0 - 10**(pH-pKa) / (1 + 10**(pH-pKa)))
        else: pqr[i,-1] = 0.0

    return pqr

In [120]:
pka_file='%s.pka' % PDB_id
pH=7.0

pcr=partial_charges(pka_file,pH)

sel=pcr[np.where(np.isin(pcr[:,2],['A']))]

print(sel[:,-1].astype('float'))

print('\nTotal charge at pH = %.2f is Q = %.2f' % (pH,np.sum(pcr[np.where(pcr[:,2]=='A')][:,-1].astype('float'))))

[-0.99  -0.99  -0.99  -0.99  -0.99  -0.99  -0.99  -0.99  -0.99  -0.99
 -0.99  -0.99  -0.99  -0.99  -0.99  -0.91  -0.81  -0.98  -0.99  -0.99
 -0.99  -0.99  -0.99  -0.99  -0.99  -0.99  -0.99  -0.99  -0.99  -0.99
 -0.99  -0.99  -0.99  -0.99  -0.99  -0.99  -0.99  -0.99  -0.99  -0.99
 -0.99  -0.93  -0.99  -0.94  -0.99  -0.99  -0.99  -0.99  -0.99  -0.99
 -0.99  -0.99  -0.99  -0.97  -0.99  -0.99  -0.99  -0.99  -0.99  -0.98
 -0.99  -0.99  -0.99  -0.98  -0.99  -0.83  -0.99  -0.99  -0.99  -0.99
 -0.99  -0.99  -0.99  -0.91  -0.99  -0.99  -0.99  -0.99  -0.99  -0.99
 -0.99  -0.99  -0.99  -0.99  -0.99  -0.99  -0.94  -0.99  -0.99  -0.99
 -0.99  -0.99  -0.99  -0.99  -0.99  -0.99  -0.99  -0.99  -0.99   0.166
  0.261  0.013  0.     0.215  0.86   0.494  0.     0.     0.94   0.806
  0.009  0.442  0.208  0.013  0.098  0.08  -0.    -0.    -0.    -0.
 -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.
 -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.
 -0.    -0.    -0.    -0

### Convert to XYZ based on propka-determined pKa

In [53]:
# chains is a list of identifiers
def convert_propka(pdb_file: str, propka_file: str, output_xyz_file: str, pH=7.0, chains=None):
    """Convert propka-parsed PDB to coarse grained XYZ file; one bead per amino acid, adding sidechain beads for ionizable amino acid"""

    pqr=mda.Universe(pqr_file)
    pqr.atoms.translate(-pqr.atoms.center_of_mass()) # moves COM to [0,0,0]

    # keep only protein atoms and optionally selected chains
    # omit hydrogen atoms
    if chains: traj=pqr.select_atoms('protein and not name H* and segid %s' % ' '.join(chains))
    else: traj=pqr.select_atoms('protein and not name H*')

    # load partial charges
    pcr=partial_charges(pka_file,7.0)
    if chains: pcr=pcr[np.where(np.isin(pcr[:,2],chains))]

    residues = []
    chrdict = {}
    
    for res in traj.residues:

        name = res.resname
        com = res.atoms.center_of_mass()

        residues.append(dict(name=name,cm=com))

        # Map residue and atom names to sidechain bead names
        sidechain_map = {
            "ASP": ("Dsc", "OD1"),
            "GLU": ("Esc", "OE1"),
            "TYR": ("Tsc", "OH"),
            "ARG": ("Rsc", "CZ"),
            "LYS": ("Ksc", "NZ"),
            "HIS": ("Hsc", "NE2"),
            "CYS": ("Csc", "SG"),
        }

        chr_=0.0
        for atom in res.atoms: chr_+=np.float32(atom.charge)

        #chr_ = res.atoms.total_charge()

        # consider only charges above some cutoff
        if abs(chr_) > 1e-3:

            bead_name, atom_name = sidechain_map.get(name,(None,None))
            
            if bead_name:

                bn = '%s%s%i' % (bead_name,res.segid,res.resid)
                residues.append(dict(name=bn, cm=traj.select_atoms('resid %i and name %s' % (res.resid,atom_name)).positions[0]))

            else:

                bn = 'TRC%s%i' % (res.segid,res.resid)
                residues.append(dict(name=bn, cm=res.atoms.center_of_charge())) # we have a terminal charge

            chridx += 1

            chrdict[bn] = chr_

    with open(output_xyz_file, "w") as f:
        
        f.write(f"{len(residues)}\n")
        f.write(
            f"Converted with Duello pqr2xyz.py with {pqr_file} (https://github.com/mlund/pdb2xyz)\n"
        )
        
        for i in residues:
            
            f.write(f"{i['name']} {i['cm'][0]:.3f} {i['cm'][1]:.3f} {i['cm'][2]:.3f}\n")
        logging.info(
            f"Converted {pqr_file} -> {output_xyz_file} with {len(residues)} residues."
        )

    return chrdict

NameError: name 'os' is not defined

### Convert PDB to PQR

In [2]:
PDB_id='4F5S'
pdb_file='../tests/%s.pdb' % PDB_id

In [3]:
%%capture --no-stdout

#if needed at this stage, split into chains

from Bio.PDB import PDBParser
from Bio.PDB.PDBIO import PDBIO

parser = PDBParser()
io = PDBIO()

structure = parser.get_structure(PDB_id, pdb_file)
pdb_chains = structure.get_chains()
for chain in pdb_chains:
    io.set_structure(chain)
    io.save('../tests/' + structure.get_id() + "_" + chain.get_id() + ".pdb")

In [7]:
%%capture --no-stdout

pH=7.0
pqr_file='../tests/%s_pH%.1f.pqr' % (PDB_id,pH)

pqr=run_pdb2pqr([pdb_file,pqr_file,
                 '--keep-chain',
                 '--titration-state-method=propka',
                 '--with-ph=%.2f' % pH,
                 '--ff=PARSE',
                 '--drop-water',
                 '-q',
                 '--whitespace'])

pH=5.0
pqr_file='../tests/%s_pH%.1f.pqr' % (PDB_id,pH)

pqr=run_pdb2pqr([pdb_file,pqr_file,
                 '--keep-chain',
                 '--titration-state-method=propka',
                 '--with-ph=%.2f' % pH,
                 '--ff=PARSE',
                 '--drop-water',
                 '-q',
                 '--whitespace'])

In [8]:
#we do this also only for a monomer (chain A)

PDB_id='4F5S_A'
pdb_file='../tests/%s.pdb' % PDB_id

In [9]:
%%capture --no-stdout

pH=7.0
pqr_file='../tests/%s_pH%.1f.pqr' % (PDB_id,pH)

pqr=run_pdb2pqr([pdb_file,pqr_file,
                 '--keep-chain',
                 '--titration-state-method=propka',
                 '--with-ph=%.2f' % pH,
                 '--ff=PARSE',
                 '--drop-water',
                 '-q',
                 '--whitespace'])

pH=5.0
pqr_file='../tests/%s_pH%.1f.pqr' % (PDB_id,pH)

pqr=run_pdb2pqr([pdb_file,pqr_file,
                 '--keep-chain',
                 '--titration-state-method=propka',
                 '--with-ph=%.2f' % pH,
                 '--ff=PARSE',
                 '--drop-water',
                 '-q',
                 '--whitespace'])

### Convert PQR to XYZ

In [34]:
def convert_pqr(pqr_file: str, output_xyz_file: str, chains=None):
    """Convert PQR to coarse grained XYZ file; one bead per amino acid, adding sidechain beads for ionizable amino acid"""

    pqr=mda.Universe(pqr_file)
    pqr.atoms.translate(-pqr.atoms.center_of_mass()) # moves COM to [0,0,0]

    # keep only protein atoms and optionally selected chains
    # omit hydrogen atoms
    if chains: traj=pqr.select_atoms('protein and not name H* and segid %s' % chains)
    else: traj=pqr.select_atoms('protein and not name H*')

    residues = []
    chrdict = {}
    chridx = 0
    
    for res in traj.residues:

        name = res.resname
        com = res.atoms.center_of_mass()

        residues.append(dict(name=name,cm=com))

        # Map residue and atom names to sidechain bead names
        sidechain_map = {
            "ASP": ("Dsc", "OD1"),
            "GLU": ("Esc", "OE1"),
            "TYR": ("Tsc", "OH"),
            "ARG": ("Rsc", "CZ"),
            "LYS": ("Ksc", "NZ"),
            "HIS": ("Hsc", "NE2"),
            "CYS": ("Csc", "SG"),
        }

        chr_=0.0
        for atom in res.atoms: chr_+=np.float32(atom.charge)

        #chr_ = res.atoms.total_charge()

        # consider only charges above some cutoff
        if abs(chr_) > 1e-3:

            bead_name, atom_name = sidechain_map.get(name,(None,None))
            
            if bead_name:

                bn = '%s%s%i' % (bead_name,res.segid,res.resid)
                residues.append(dict(name=bn, cm=traj.select_atoms('resid %i and name %s' % (res.resid,atom_name)).positions[0]))

            else:

                bn = 'TRC%s%i' % (res.segid,res.resid)
                residues.append(dict(name=bn, cm=res.atoms.center_of_charge())) # we have a terminal charge

            chridx += 1

            chrdict[bn] = chr_

    with open(output_xyz_file, "w") as f:
        
        f.write(f"{len(residues)}\n")
        f.write(
            f"Converted with Duello pqr2xyz.py with {pqr_file} (https://github.com/mlund/pdb2xyz)\n"
        )
        
        for i in residues:
            
            f.write(f"{i['name']} {i['cm'][0]:.3f} {i['cm'][1]:.3f} {i['cm'][2]:.3f}\n")
        logging.info(
            f"Converted {pqr_file} -> {output_xyz_file} with {len(residues)} residues."
        )

    return chrdict

def write_topology():

    #with open('topology.yaml', "w") as f:
with open('BSA-full2.yaml', "w") as f:
    f.write(
"""comment: "Calvados 3 coarse grained amino acid model for use with Duello / Faunus"
pH: %.2f
version: 0.1.0
atoms:""" % 7.20
)
    for key in cdr.keys(): f.write(
"""
  - {charge: %.2f, hydrophobicity: !Lambda 0, mass: 0, name: %s, σ: 2.0, ε: 0.8368}""" % (cdr[key],key)
    )
    f.write(
"""
  - {charge: 0.0, hydrophobicity: !Lambda 0.7407902764839954, mass: 156.19, name: ARG, σ: 6.56, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.092587557536158,  mass: 115.09, name: ASP, σ: 5.58, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.000249590539426,  mass: 129.11, name: GLU, σ: 5.92, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.1380602542039267, mass: 128.17, name: LYS, σ: 6.36, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.4087176216525476, mass: 137.14, name: HIS, σ: 6.08, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.3706962163690402, mass: 114.1,  name: ASN, σ: 5.68, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.3143449791669133, mass: 128.13, name: GLN, σ: 6.02, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.4473142572693176, mass: 87.08,  name: SER, σ: 5.18, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.7538308115197386, mass: 57.05,  name: GLY, σ: 4.5,  ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.2672387936544146, mass: 101.11, name: THR, σ: 5.62, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.3377244362031627, mass: 71.07,  name: ALA, σ: 5.04, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.5170874160398543, mass: 131.2,  name: MET, σ: 6.18, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.950628687301107,  mass: 163.18, name: TYR, σ: 6.46, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.2936174211771383, mass: 99.13,  name: VAL, σ: 5.86, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 1.033450123574512,  mass: 186.22, name: TRP, σ: 6.78, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.5548615312993875, mass: 113.16, name: LEU, σ: 6.18, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.5130398874425708, mass: 113.16, name: ILE, σ: 6.18, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.3469777523519372, mass: 97.12,  name: PRO, σ: 5.56, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.8906449355499866, mass: 147.18, name: PHE, σ: 6.36, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.5922529084601322, mass: 103.14, name: CYS, σ: 5.48, ε: 0.8368}
"""
)
    f.write(
"""
system:
  energy:
    nonbonded:
      # Note that a Coulomb term is automatically added, so don't specify one here!
      default:
        - !AshbaughHatch {mixing: arithmetic, cutoff: 20.0}
"""
)

In [35]:
PQR_id='4F5S'
pH=7.0
pqr_file='../tests/%s_pH%.1f.pqr' % (PQR_id,pH)
xyz_file='../tests/%s_pqr_pH%.1f.xyz' % (PQR_id,pH)

In [36]:
cdict=convert_pqr(pqr_file,xyz_file)

In [37]:
cdict

{'KscA4': np.float32(1.0000001),
 'EscA6': np.float32(-1.0000001),
 'RscA10': np.float32(0.9999999),
 'KscA12': np.float32(1.0000001),
 'DscA13': np.float32(-1.0000001),
 'EscA16': np.float32(-1.0000001),
 'EscA17': np.float32(-1.0000001),
 'KscA20': np.float32(1.0000001),
 'DscA37': np.float32(-1.0000001),
 'EscA38': np.float32(-1.0000001),
 'KscA41': np.float32(1.0000001),
 'EscA45': np.float32(-1.0000001),
 'EscA48': np.float32(-1.0000001),
 'KscA51': np.float32(1.0000001),
 'DscA56': np.float32(-1.0000001),
 'EscA57': np.float32(-1.0000001),
 'EscA63': np.float32(-1.0000001),
 'KscA64': np.float32(1.0000001),
 'HscA67': np.float32(1.0),
 'DscA72': np.float32(-1.0000001),
 'EscA73': np.float32(-1.0000001),
 'KscA76': np.float32(1.0000001),
 'RscA81': np.float32(0.9999999),
 'EscA82': np.float32(-1.0000001),
 'DscA86': np.float32(-1.0000001),
 'DscA89': np.float32(-1.0000001),
 'EscA92': np.float32(-1.0000001),
 'KscA93': np.float32(1.0000001),
 'EscA95': np.float32(-1.0000001),
 'Es

In [107]:
def calvados_template():
    return """
{%- set f = 1.0 - sidechains -%}
{%- set zCTR = - 10**(pH-3.16) / (1 + 10**(pH-3.16)) -%}
{%- set zASP = - 10**(pH-3.43) / (1 + 10**(pH-3.43)) -%}
{%- set zGLU = - 10**(pH-4.14) / (1 + 10**(pH-4.14)) -%}
{%- set zCYS = 10**(pH-6.25) / (1 + 10**(pH-6.25)) -%}
{%- set zHIS = 1 - 10**(pH-6.45) / (1 + 10**(pH-6.45)) -%}
{%- set zNTR = 1 - 10**(pH-7.64) / (1 + 10**(pH-7.64)) -%}
{%- set zLYS = 1 - 10**(pH-10.68) / (1 + 10**(pH-10.68)) -%}
{%- set zARG = 1 - 10**(pH-12.5) / (1 + 10**(pH-12.5)) -%}
comment: "Calvados 3 coarse grained amino acid model for use with Duello / Faunus"
pH: {{ pH }}
sidechains: {{ sidechains }}
version: 0.1.0
atoms:
  - {charge: {{ "%.2f" % zCTR }}, hydrophobicity: !Lambda 0, mass: 0, name: CTR, σ: 2.0, ε: 0.8368}
  - {charge: {{ "%.2f" % zNTR }}, hydrophobicity: !Lambda 0, mass: 0, name: NTR, σ: 2.0, ε: 0.8368}
{%- if sidechains %}
  - {charge: {{ "%.2f" % zGLU }}, hydrophobicity: !Lambda 0, mass: 0, name: Esc, σ: 2.0, ε: 0.8368}
  - {charge: {{ "%.2f" % zASP }}, hydrophobicity: !Lambda 0, mass: 0, name: Dsc, σ: 2.0, ε: 0.8368}
  - {charge: {{ "%.2f" % zHIS }}, hydrophobicity: !Lambda 0, mass: 0, name: Hsc, σ: 2.0, ε: 0.8368}
  - {charge: {{ "%.2f" % zARG }}, hydrophobicity: !Lambda 0, mass: 0, name: Rsc, σ: 2.0, ε: 0.8368}
  - {charge: {{ "%.2f" % zLYS }}, hydrophobicity: !Lambda 0, mass: 0, name: Ksc, σ: 2.0, ε: 0.8368}
  - {charge: {{ "%.2f" % zCYS }}, hydrophobicity: !Lambda 0, mass: 0, name: Csc, σ: 2.0, ε: 0.8368}
{%- endif %}
  - {charge: {{ "%.2f" % (zARG * f) }}, hydrophobicity: !Lambda 0.7407902764839954, mass: 156.19, name: ARG, σ: 6.56, ε: 0.8368, custom: {alpha: {{ f * alpha }}}}
  - {charge: {{ "%.2f" % (zASP * f) }}, hydrophobicity: !Lambda 0.092587557536158,  mass: 115.09, name: ASP, σ: 5.58, ε: 0.8368, custom: {alpha: {{ f * alpha }}}}
  - {charge: {{ "%.2f" % (zGLU * f) }}, hydrophobicity: !Lambda 0.000249590539426,  mass: 129.11, name: GLU, σ: 5.92, ε: 0.8368, custom: {alpha: {{ f * alpha }}}}
  - {charge: {{ "%.2f" % (zLYS * f) }}, hydrophobicity: !Lambda 0.1380602542039267, mass: 128.17, name: LYS, σ: 6.36, ε: 0.8368, custom: {alpha: {{ f * alpha }}}}
  - {charge: {{ "%.2f" % (zHIS * f) }}, hydrophobicity: !Lambda 0.4087176216525476, mass: 137.14, name: HIS, σ: 6.08, ε: 0.8368, custom: {alpha: {{ f * alpha }}}}
  - {charge: 0.0, hydrophobicity: !Lambda 0.3706962163690402, mass: 114.1,  name: ASN, σ: 5.68, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.3143449791669133, mass: 128.13, name: GLN, σ: 6.02, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.4473142572693176, mass: 87.08,  name: SER, σ: 5.18, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.7538308115197386, mass: 57.05,  name: GLY, σ: 4.5,  ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.2672387936544146, mass: 101.11, name: THR, σ: 5.62, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.3377244362031627, mass: 71.07,  name: ALA, σ: 5.04, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.5170874160398543, mass: 131.2,  name: MET, σ: 6.18, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.950628687301107,  mass: 163.18, name: TYR, σ: 6.46, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.2936174211771383, mass: 99.13,  name: VAL, σ: 5.86, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 1.033450123574512,  mass: 186.22, name: TRP, σ: 6.78, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.5548615312993875, mass: 113.16, name: LEU, σ: 6.18, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.5130398874425708, mass: 113.16, name: ILE, σ: 6.18, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.3469777523519372, mass: 97.12,  name: PRO, σ: 5.56, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.8906449355499866, mass: 147.18, name: PHE, σ: 6.36, ε: 0.8368}
  - {charge: {{ "%.2f" % (zCYS * f) }}, hydrophobicity: !Lambda 0.5922529084601322, mass: 103.14, name: CYS, σ: 5.48, ε: 0.8368, custom: {alpha: {{ f * alpha }}}}
  - {charge: 0.0, hydrophobicity: !Lambda 0.5922529084601322, mass: 103.14, name: CSS, σ: 5.48, ε: 0.8368}

system:
  energy:
    nonbonded:
      # Note that a Coulomb term is automatically added, so don't specify one here!
      default:
        - !AshbaughHatch {mixing: arithmetic, cutoff: 20.0}
"""

In [4]:
#cdr=convert_pqr('/Users/anze/Sandbox/BSA-chainA-pH7.20.pqr', 'test.xyz', chains='A')
cdr=convert_pqr('/Users/anze/Sandbox/BSA-full-pH7.20.pqr', 'BSA-full2.xyz')

In [87]:
pqr=run_pdb2pqr(['/Users/anze/Sandbox/BSA_9QQD.pdb','BSA-2.pqr','--keep-chain','--titration-state-method=propka','--with-ph=7.20','--ff=PARSE','--drop-water','-q','--whitespace'])

Error parsing line: invalid literal for int() with base 10: ''
<CRYST1    1.000    1.000    1.000  90.00  90.00  90.00 P 1>
Truncating remaining errors for record type:CRYST1

['CRYST1']
Missing atom OXT in residue ALA A 581
Missing atom OXT in residue ALA A 581
Missing atom OXT in residue ALA A 581
Ignoring 467 header lines in output.


In [5]:
#with open('topology.yaml', "w") as f:
with open('BSA-full2.yaml', "w") as f:
    f.write(
"""comment: "Calvados 3 coarse grained amino acid model for use with Duello / Faunus"
pH: %.2f
version: 0.1.0
atoms:""" % 7.20
)
    for key in cdr.keys(): f.write(
"""
  - {charge: %.2f, hydrophobicity: !Lambda 0, mass: 0, name: %s, σ: 2.0, ε: 0.8368}""" % (cdr[key],key)
    )
    f.write(
"""
  - {charge: 0.0, hydrophobicity: !Lambda 0.7407902764839954, mass: 156.19, name: ARG, σ: 6.56, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.092587557536158,  mass: 115.09, name: ASP, σ: 5.58, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.000249590539426,  mass: 129.11, name: GLU, σ: 5.92, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.1380602542039267, mass: 128.17, name: LYS, σ: 6.36, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.4087176216525476, mass: 137.14, name: HIS, σ: 6.08, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.3706962163690402, mass: 114.1,  name: ASN, σ: 5.68, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.3143449791669133, mass: 128.13, name: GLN, σ: 6.02, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.4473142572693176, mass: 87.08,  name: SER, σ: 5.18, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.7538308115197386, mass: 57.05,  name: GLY, σ: 4.5,  ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.2672387936544146, mass: 101.11, name: THR, σ: 5.62, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.3377244362031627, mass: 71.07,  name: ALA, σ: 5.04, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.5170874160398543, mass: 131.2,  name: MET, σ: 6.18, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.950628687301107,  mass: 163.18, name: TYR, σ: 6.46, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.2936174211771383, mass: 99.13,  name: VAL, σ: 5.86, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 1.033450123574512,  mass: 186.22, name: TRP, σ: 6.78, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.5548615312993875, mass: 113.16, name: LEU, σ: 6.18, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.5130398874425708, mass: 113.16, name: ILE, σ: 6.18, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.3469777523519372, mass: 97.12,  name: PRO, σ: 5.56, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.8906449355499866, mass: 147.18, name: PHE, σ: 6.36, ε: 0.8368}
  - {charge: 0.0, hydrophobicity: !Lambda 0.5922529084601322, mass: 103.14, name: CYS, σ: 5.48, ε: 0.8368}
"""
)
    f.write(
"""
system:
  energy:
    nonbonded:
      # Note that a Coulomb term is automatically added, so don't specify one here!
      default:
        - !AshbaughHatch {mixing: arithmetic, cutoff: 20.0}
"""
)

Using PQR:
* automatically takes care of N-terminal and C-terminal charges
* automatically takes care of uncharged SS CYS residues

While Duello:
* Can assign double charge if the N-terminal AA has an ionizable side group

Relevant atoms for sidegroups:
* LYS: HZ*

In [40]:
sidechain_map = {
            ("ASP"): ("OD1", "Dsc"),
            ("GLU", "OE1"): "Esc",
            ("TYR", "CZ"): "Tsc",
            ("ARG", "CZ"): "Rsc",
            ("LYS", "NZ"): "Ksc",
            ("HIS", "NE2"): "Hsc",
            ("CYS", "SG"): "Csc",
}

#bead_name = sidechain_map.get((res.name, atom.name))

In [42]:
sidechain_map.get("ASP")[0]

'OD1'

In [81]:
traj.select_atoms('resid 573 and name NZ')

<AtomGroup with 1 atom>

In [62]:
sl.positions

array([[-3.444, 33.006, 83.23 ]], dtype=float32)